In [1]:
import sys
import os
import re
import numpy as np
import pandas as pd
import scipy

from extract_scalar import *
from extract_numbers import *

In [2]:
ds_root = '/volumes/hyijie_psy/CPP/data_mid-level_2/data_raw'
tgt_dir='..'

results_dir_mem = '../results/joint-modeling_mem'
results_dir_perc = '../results/joint-modeling_perc'

dir_ddm_mem=os.path.join(tgt_dir, 'results', 'ddm', 'mem')
if not os.path.exists(dir_ddm_mem):
    os.makedirs(dir_ddm_mem)

dir_ddm_perc=os.path.join(tgt_dir, 'results', 'ddm', 'perc')
if not os.path.exists(dir_ddm_perc):
    os.makedirs(dir_ddm_perc)

In [3]:
# 这个cell是为了mem的ddm——保留版，这个cell没有提取欧式距离
sub_dir = [f'ACC{i:03}' for i in range(14,37)]
data_ddm_behavior_mem = []
for sub in sub_dir:
    data_behavior = scipy.io.loadmat(os.path.join(ds_root, sub, sub+'forDMA.mat'))['datOut']

    data_behavior_mem = data_behavior[(data_behavior[:,0] == 2),:]
    data_behavior_mem = data_behavior_mem.astype(object)
    data_behavior_mem[:,0] = 'mem'

    data_ddm_behavior_mem.append(data_behavior_mem)

data_ddm_behavior_mem = np.concatenate(data_ddm_behavior_mem,axis=0)
data_ddm_behavior_mem = pd.DataFrame(data_ddm_behavior_mem, columns=['type', 'accuracy', 'rt'])

data_feature_mem = pd.read_csv(os.path.join(results_dir_mem,'joint_modeling_all_mem.csv'))

data4ddm_mem_final = pd.concat([data_ddm_behavior_mem, data_feature_mem], axis=1)

save_path = os.path.join(dir_ddm_mem, 'data4ddm_mem.csv')
data4ddm_mem_final.to_csv(save_path, index=False)


In [3]:
# 这个cell是为了从events中提出behavior相关数据_mem
sub_dir = [f'ACC{i:03}' for i in range(14,37)]
data_ddm_behavior_mem = []
for sub in sub_dir:
    data_behavior = scipy.io.loadmat(os.path.join(ds_root, sub, 'events.mat'))['events']

    behavior_df = pd.DataFrame({
        'mode': [extract_scalar(x) for x in data_behavior['mode'][0]], 
        'type': [extract_scalar(x) for x in data_behavior['type'][0]],  
        'rt': [extract_scalar(x) for x in data_behavior['rt'][0]],       
        'item_1':[extract_scalar(x) for x in data_behavior['item1'][0]],
        'item_2':[extract_scalar(x) for x in data_behavior['item2'][0]],
        'item_cue':[extract_scalar(x) for x in data_behavior['cueitem'][0]],
        'accuracy':[extract_scalar(x) for x in data_behavior['correct'][0]]
        })
    
    behavior_df = behavior_df[(behavior_df['mode'] == 'mem') & (behavior_df['type'] == 'resp')]

    behavior_df['rt'] = behavior_df['rt']/1000
    behavior_df['item_1_number'] = behavior_df['item_1'].apply(extract_numbers)
    behavior_df['item_2_number'] = behavior_df['item_2'].apply(extract_numbers)
    behavior_df['item_cue_number'] = behavior_df['item_cue'].apply(extract_numbers)

    # 计算第一张脸与cue的欧式距离
    behavior_df['distance_1_cue'] = behavior_df.apply(
        lambda row: np.sqrt(np.sum((np.array(row['item_1_number']) - np.array(row['item_cue_number'])) ** 2)),
        axis=1
    )

    # 计算第二张脸与cue的欧式距离
    behavior_df['distance_2_cue'] = behavior_df.apply(
        lambda row: np.sqrt(np.sum((np.array(row['item_2_number']) - np.array(row['item_cue_number'])) ** 2)),
        axis=1
    )

    # 相加
    behavior_df['distance_sum'] = behavior_df['distance_1_cue']+behavior_df['distance_2_cue']
    
    data_ddm_behavior_mem.append(behavior_df)

data_ddm_behavior_mem = np.concatenate(data_ddm_behavior_mem,axis=0)
data_ddm_behavior_mem = pd.DataFrame(data_ddm_behavior_mem, columns=['type', 'accuracy', 'rt', 'item1', 'item2', 'itemcue', 'correct',' item1_number', 'item2_number', 'item_cue_number', 'distance_1_cue', 'distance_2_cue','distance_sum'])

data_feature_mem = pd.read_csv(os.path.join(results_dir_mem,'joint_modeling_all_mem.csv'))

data4ddm_mem_final = pd.concat([data_ddm_behavior_mem, data_feature_mem], axis=1)

save_path = os.path.join(dir_ddm_mem, 'data4ddm_mem.csv')
data4ddm_mem_final.to_csv(save_path, index=False)

In [4]:
# 这个cell是为了从events中提出behavior相关数据_perc
sub_dir = [f'ACC{i:03}' for i in range(14,37)]
data_ddm_behavior_perc = []
for sub in sub_dir:
    data_behavior = scipy.io.loadmat(os.path.join(ds_root, sub, 'events.mat'))['events']

    behavior_df = pd.DataFrame({
        'mode': [extract_scalar(x) for x in data_behavior['mode'][0]], 
        'type': [extract_scalar(x) for x in data_behavior['type'][0]],  
        'rt': [extract_scalar(x) for x in data_behavior['rt'][0]],       
        'item_1':[extract_scalar(x) for x in data_behavior['item1'][0]],
        'item_2':[extract_scalar(x) for x in data_behavior['item2'][0]],
        'accuracy':[extract_scalar(x) for x in data_behavior['correct'][0]]})
    
    behavior_df['rt'] = behavior_df['rt']/1000
    behavior_df['item_1_number'] = behavior_df['item_1'].apply(extract_numbers)
    behavior_df['item_2_number'] = behavior_df['item_2'].apply(extract_numbers)

    # 计算欧式距离
    behavior_df['distance'] = behavior_df.apply(
        lambda row: np.sqrt(np.sum((np.array(row['item_1_number']) - np.array(row['item_2_number'])) ** 2)),
        axis=1
    )

    data_behavior_perc = behavior_df[(behavior_df['mode'] == 'perc') & (behavior_df['type'] == 'resp')]
    
    data_ddm_behavior_perc.append(data_behavior_perc)

data_ddm_behavior_perc = np.concatenate(data_ddm_behavior_perc,axis=0)
data_ddm_behavior_perc = pd.DataFrame(data_ddm_behavior_perc, columns=['type', 'accuracy', 'rt', 'item1', 'item2', 'correct',' item1_number', 'item2_number', 'distance'])

data_feature_perc = pd.read_csv(os.path.join(results_dir_perc,'joint_modeling_all_perc.csv'))

data4ddm_perc_final = pd.concat([data_ddm_behavior_perc, data_feature_perc], axis=1)

save_path = os.path.join(dir_ddm_perc, 'data4ddm_perc.csv')
data4ddm_perc_final.to_csv(save_path, index=False)